# PREPARATIVOS

In [1]:
import pandas as pd
import numpy as np
import itertools as it
import re
import os
from funciones_1 import *
from funciones_2 import *
from importlib import reload
import time
import matplotlib.pyplot as plt
#import warnings
#warnings.filterwarnings("error")

In [2]:
%load_ext line_profiler

In [3]:
#FBL3N = leer_excel("FBL3N.xlsx")

In [4]:
#FBL5N = leer_excel("FBL5N.xlsx")

In [5]:
#KNA1 = leer_excel("KNA1.xlsx")

In [6]:
#%store FBL3N FBL5N KNA1

In [7]:
%store -r FBL3N FBL5N KNA1

In [8]:
#PROPUESTA = leer_excel("PROPUESTA.xlsx", None)

# PARTE 1

In [9]:
textos = FBL3N["Texto"]
FBL3N["RUT"] = textos.apply(detectar_rut)
FBL3N["N° Factura"] = textos.apply(detectar_n_factura)

RUT_FBL3N = FBL3N[pd.notna(FBL3N["RUT"])]
FACT_FBL3N = FBL3N[pd.isna(FBL3N["RUT"]) & pd.notna(FBL3N["N° Factura"])]

# PARTE 2

### Trabajar con calces de factura

In [10]:
FBL5N["Referencia"] = FBL5N["Referencia"].apply(lambda s: str(s))
FBL5N["Cuenta"] = FBL5N["Cuenta"][pd.notna(FBL5N["Cuenta"])].apply(lambda n: str(int(n)))

KNA1["Nº ident.fis.1"] = KNA1["Nº ident.fis.1"].apply(lambda s: str(s).lstrip("0"))
KNA1["Cliente"] = KNA1["Cliente"][pd.notna(KNA1["Cliente"])].apply(lambda s: str(int(s)))

In [11]:
# Calzar números de factura
filtro_fact = FBL5N["Referencia"].isin(FACT_FBL3N["N° Factura"])
FACT_FBL5N = FBL5N[filtro_fact]
NO_FACT_FBL5N = FBL5N[~filtro_fact]

In [12]:
# Calzar RUTs
RUT_ID_FBL3N = pd.merge(RUT_FBL3N, KNA1, left_on="RUT", right_on="Nº ident.fis.1")

filtro_id5 = NO_FACT_FBL5N["Cuenta"].isin(RUT_ID_FBL3N["Cliente"])
ID_FBL5N = NO_FACT_FBL5N[filtro_id5]
NN_FBL5N = NO_FACT_FBL5N[~filtro_id5]

filtro_id3 = RUT_ID_FBL3N["Cliente"].isin(ID_FBL5N["Cuenta"])
ID_FBL3N = RUT_ID_FBL3N[filtro_id3]
SOLO_RUT_FBL3N = RUT_ID_FBL3N[filtro_id3]

In [13]:
try:
    os.mkdir("RESULTADOS/")
except:
    pass

In [14]:
MATCHES_FACT = pd.merge(FACT_FBL3N, FACT_FBL5N, left_on="N° Factura", right_on="Referencia")
escribir_excel(FACT_FBL5N, "RESULTADOS/Facturas.xlsx")

Escribiendo nuevo archivo Excel: RESULTADOS/Facturas.xlsx
Archivo Excel escrito.


# PARTE 3

## Matches

In [15]:
DF, D = calzar(ID_FBL3N, ID_FBL5N)

In [16]:
DF

,Grupo,Soc.,Nº documento,Clase de documento,Fecha de documento,Importe en moneda local,Moneda local,Texto,Cliente,Nº ident.fis.1,Nombre 1,Asignación,Ejerc./mes,Nº doc.,Referencia,Cla,Importe en ML,Mon.
0,1,2000,100018445.0,CT,2019-01-23,-11567,CLP,PAGO:PROVEEDORES 0969424002,1341643,96942400-2,MEGASALUD SPA,1610070236,2021/07,1.600018e+09,1850834,DG,-350181,CLP
1,1,2000,100018445.0,CT,2019-01-23,-11567,CLP,PAGO:PROVEEDORES 0969424002,1341643,96942400-2,MEGASALUD SPA,1801572657,2021/10,2.500352e+09,9842122,DR,93998,CLP
2,1,2000,100018445.0,CT,2019-01-23,-11567,CLP,PAGO:PROVEEDORES 0969424002,1341643,96942400-2,MEGASALUD SPA,1801573445,2021/10,2.500353e+09,9842526,DR,267750,CLP
3,2,2000,100057473.0,CT,2021-02-19,-1942592,CLP,PAGO:PROVEEDORES 0761419889,1726116,76141988-9,OPERACIONES INTEGRALES CHACABUCO SO,8783031,2020/06,1.400103e+09,PAGO CHILE,DZ,497452,CLP
4,2,2000,100057473.0,CT,2021-02-19,-1942592,CLP,PAGO:PROVEEDORES 0761419889,1726116,76141988-9,OPERACIONES INTEGRALES CHACABUCO SO,1801240210,2021/02,2.500053e+09,9494241,DR,1309167,CLP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,115,2000,100457436.0,CT,2021-12-03,-713461,CLP,PAGO:PROVEEDORES 0761419889,1726116,76141988-9,OPERACIONES INTEGRALES CHACABUCO SO,1801584972,2021/10,2.500363e+09,9896120,DR,99617,CLP
341,115,2000,100457436.0,CT,2021-12-03,-713461,CLP,PAGO:PROVEEDORES 0761419889,1726116,76141988-9,OPERACIONES INTEGRALES CHACABUCO SO,1801592858,2021/11,2.500371e+09,9896559,DR,613844,CLP
342,116,2000,100457463.0,CT,2021-12-03,-24048,CLP,0993010006 P.PROVEEDOR SEGU,1005767,99301000-6,SEGUROS VIDA SECURITY PREVISION S.A,1801633795,2021/11,2.500406e+09,9903950,DR,24048,CLP
343,117,2000,100457533.0,CT,2021-12-03,-106020,CLP,PAGO:PROVEEDORES 0780230304,1139847,78023030-4,SOFRUCO ALIMENTOS LIMITADA,1801565200,2021/10,2.500346e+09,9862080,DR,42773,CLP


In [ ]:
d = extraer_datos(ID_FBL3N, ID_FBL5N)
match_1_1(d)
match_1_T(d)
match_1_V(d)
%lprun -f match_1_M_2 match_1_M_2(d)

In [ ]:
escribir_excel(DF, "RESULTADOS/Todo.xlsx")

# PARTE 4

### Determinar qué datos encontré yo que Roberto no, y viceversa

1. RUT y Monto

In [ ]:
RUT_Y_MONTO = leer_excel("PROPUESTA/RUT y Monto.xlsx")

In [ ]:
outer_rym = pd.merge(DF, RUT_Y_MONTO, on="Nº documento", how="outer", indicator=True)

In [ ]:
try:
    os.mkdir("COMPARACIONES/")
except:
    pass

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "left_only"], "COMPARACIONES/RUT y Monto - LEFT.xlsx")

In [ ]:
escribir_excel(outer_rym[outer_rym["_merge"] == "right_only"], "COMPARACIONES/RUT y Monto - RIGHT.xlsx")